# Week 14 Problem 1

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select *Kernel*, and restart the kernel and run all cells (*Restart & Run all*).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select *File* → *Save and CheckPoint*)

5. When you are ready to submit your assignment, go to *Dashboard* → *Assignments* and click the *Submit* button. Your work is not submitted until you click *Submit*.

6. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

## Author: Radhir Kothuri
### Primary Reviewer: Apurv Garg

# Due Date: 6 PM, April 30, 2018

In [1]:
# Standard imports
% matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats as sts
import pymc3 as pm

# testing tools
from nose.tools import (
    assert_equal, assert_true, assert_is_instance
)

from numpy.testing import assert_almost_equal

# These two lines suppress warnings that sometimes
# occur when making visualizations
import warnings
warnings.filterwarnings('ignore')

m_true = 0.7
b_true = 0.3
def theModel(xmin=0, xmax = 1, num=20):
    np.random.seed(23)
    sigma = 0.3
    x = np.linspace(xmin, xmax, num)
    y = b_true + m_true * x - sigma * np.random.randn(len(x))
    return(x, y)

## Question 1

In this question, we will be exploring alternative methods of model fitting for linear models that we have explored in previous weeks. Specifically, in the question below, you will use the `sts.linregress` (Week 1 material) and the `statsmodel.formula.api.smf.OLS` functions in order to determine which evaluation method yields more accurate parameters. We will define a model as being more accurate if it has a higher `r-value`.

- Complete the function `get_model_params` that takes in 2 parameters: `x_vals` and `y_vals` that represent the `x` and `y` points respectively.
- Return a 2-tuple of the maximum r-value after evaluating both approaches as well as either the string `ols` or `linregress` depending on which method returned the maximum r-value.
- **Hint: Use the `rsquared` property after calling `fit()` when evaluating using ols method. You will need to take the square root of this in order to retrieve the actual r-value**.

In [2]:
def get_model_params(x_vals, y_vals):
    """
    Return the r-value of the approach that produces the maximum r-value
    
    Parameters
    ----------
    x_vals: a list of ints
    y_vals: a list of ints
    
    Returns
    -------
    2-tuple of the maximum r-value followed by the string of the method that returned
    the highest r-value
    """
    
    # YOUR CODE HERE
    import math
    sts_result = sts.linregress(x_vals, y_vals)
    r_sts = sts_result[2]
    
    
    ols_result = smf.OLS(y_vals, x_vals).fit()
    
    r_ols = ols_result.rsquared
    


    if r_sts >= math.sqrt(r_ols):
        return r_sts, 'linregress'
    
    else:
        return math.sqrt(r_ols), 'ols'
# get_model_params(x_vals, y_vals)
    

In [3]:
# Test Cases
x_vals, y_vals = theModel(num=100)
r_val, method = get_model_params(x_vals, y_vals)
assert_equal(method, 'ols')
assert_almost_equal(r_val, 0.8873103577785, decimal=4)

x_vals, y_vals = theModel(num=2000)
r_val, method = get_model_params(x_vals, y_vals)
assert_equal(method, 'ols')
assert_almost_equal(r_val, 0.891001337078, decimal=4)

x_vals, y_vals = theModel(xmin=1, xmax=2000, num=2000)
r_val, method = get_model_params(x_vals, y_vals)
assert_equal(method, 'ols')
assert_almost_equal(r_val, 0.999999721605, decimal=4)

## Question 2

In this question, we be using the `pymc3` library to create linear models and evaluate them using Bayesian Model Fitting using trace analysis.

- Complete the function `linear_model` which returns the result of the trace analysis from the function.
- Define a linear model `y = mx+b` where `m` is the slope of the function, and `b` is the y-intercept and where `x` and `y` are the respective input and output of the function.
- `m` is normally distributed with a `mu` of 1 and a `std` of 1.0
- `b` is uniformally distributed from 0 to 1
- For observation errrors, define a stochastic variable, `sigma`, that is normally distributed with a `mu` of 1 and a `std` of 2.0
- **Note: Please use `b`, `m`, `sigma` as the specific names for each of the distributions. This will be checked in the tests.**
- Estimate the model paramters with the maximum a posteriori (MAP) method (using default parameters).
- Use the No-U-Turn Sampler (NUTS) to generate posterior samples.
- The function will take 4 parameters: `x_vals`, `y_vals`, `random_seed`, and `n_samples`
- `random_seed` and `n_samples` should be used as parameters to the `sample()` function.
- **Note: Validation might take a little longer for this assignment, so please be patient and please start early on this problem to not overload the course server the day before it's due.**

In [6]:
def linear_model(x_vals, y_vals, random_seed, n_samples):
    '''    
    Return the result of the trace analysis
    
    Parameters
    ----------
    x_vals: A np.ndarray
    y_vals: A np.ndarray
    random_seed: An int
    n_samples: An int
    
    Returns
    -------
    A pm.backends.base.MultiTrace instance
    '''
    
    # YOUR CODE HERE
    with pm.Model() as linear_model:
        b = pm.Uniform('b', lower = 0, upper = 1)
        
        m = pm.Normal('m', mu=1.0, sd=1.0)

        sigma = pm.Normal('sigma', mu = 1, sd = 2.0)
        
        y_exp = b + m*x_vals
        
        likelihood = pm.Normal('yn', mu=y_exp, sd=sigma, observed=y_vals)
        
        start = pm.find_MAP()
        step = pm.NUTS(scaling=start)
        trace = pm.sample(n_samples, step=step,start=start, random_seed=random_seed)
    
    return trace

In [7]:
x_vals, y_vals = theModel(num=2000)
trace = linear_model(x_vals, y_vals, n_samples=2000, random_seed=23)
assert_is_instance(trace, pm.backends.base.MultiTrace)
assert_true('b' in trace.varnames)
assert_true('m' in trace.varnames)
assert_true('sigma' in trace.varnames)
for v in trace.varnames:
    assert_equal(len(trace[v]), 2000)
assert_almost_equal(trace['b'][0], 0.328787056433, decimal = 4)
assert_almost_equal(trace['m'][27], 0.703106549777, decimal=4)
assert_almost_equal(trace['sigma'][1002], 0.297812937265, decimal=4)

Optimization terminated successfully.
         Current function value: 424.411086
         Iterations: 15
         Function evaluations: 29
         Gradient evaluations: 25


100%|██████████| 2500/2500 [00:03<00:00, 626.63it/s]
